### Install Dependencies
Make sure you have the required dependencies installed. You can do this by running the following command in your terminal:

In [1]:
%pip install --only-binary=:all: tiktoken
%pip install --upgrade pip setuptools wheel
%pip install tiktoken --only-binary=:all:

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.9 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.1 MB/s  0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Install required libraries
%pip install -qU \
    langchain==0.3.* \
    langchain_openai==0.3.* \
    langchain_community \
    unstructured[md]==0.17.* \
    langgraph==0.4.* \
    websockets==15.0.*

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.utilities.requests import RequestsWrapper
from langchain_community.agent_toolkits.openapi import planner
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
import requests
import os
import json
import websockets
import requests

from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
if (openai_api_key):
    print("------------------- Loaded Key -------------------")
# print(openai_api_key)  # optional: check it loaded correctly

# Load the OpenAPI specification from the running ticketing system
root = "http://localhost:3000"
api_spec_url = f"{root}/api/docs/openapi.json"

# Download and parse the OpenAPI spec
response = requests.get(api_spec_url)
data = response.json()
data['servers'] = [{'url': root}]
openapi_spec = reduce_openapi_spec(data, dereference=False)

### Try to get it to categorize all tickets
requests_wrapper = RequestsWrapper()
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.0)

agent = planner.create_openapi_agent(
    api_spec=openapi_spec,
    requests_wrapper=requests_wrapper,
    llm=llm,
    verbose=True,
    allow_dangerous_requests=True,
    handle_parsing_errors=True,
    allow_operations=['GET', 'POST', 'PUT', 'PATCH', 'DELETE']
)

response = requests.get(root+ "/api/tickets")
tickets = response.json()

WS_URL = "ws://localhost:3000/ws"
# This async function connects to the WebSocket and listens for ticket updates
# Once a ticket update is received, it yields it for processing.
async def listen_for_ticket_updates():
    print("\n------------------- Starting Connection and Agent -------------------")
    # Establish a connection to the WebSocket server
    async with websockets.connect(WS_URL) as websocket:
        print("WebSocket connection established.")
        try:
            # Keep listening for messages from the server
            while True:
                message = await websocket.recv()  # Wait for a new message
                yield json.loads(message)
        except websockets.ConnectionClosed:
            print("WebSocket connection closed.")
        except Exception as e:
            print(f"WebSocket error: {e}")

import asyncio

# --- Async wrapper for the agent ---
async def run_agent_async(instructions: str):
    """Run the agent in a non-blocking way."""
    # agent.invoke is synchronous, so run it in a thread
    return await asyncio.to_thread(agent.invoke, instructions)

# --- Async agent helper ---
async def run_agent(ticket_id, action_description, instructions):
    """Run an agent action asynchronously with logging."""
    print(f'{action_description} for ticket: {ticket_id}')
    return await run_agent_async(instructions.strip())


# --- Simplified async ticket functions ---
async def determine_category(ticket_id):
    await run_agent(
        ticket_id,
        "Categorizing ticket",
        f"""
Based on context from this ticket {ticket_id} give it one category from: Mechanical, Quality, Maintenance, Technical, Awaiting Details.
Use “Awaiting Details” ONLY if insufficient info exists. Then POST the assigned category.
"""
    )

async def determine_priority(ticket_id):
    await run_agent(
        ticket_id,
        "Determining ticket priority",
        f"""
Assign this ticket {ticket_id} a priority: High, Medium, or Low. 
If the category is “Awaiting Details,” set priority to Low. Otherwise, POST the assigned priority.
"""
    )

async def determine_response(ticket_id):
    await run_agent(
        ticket_id,
        "Checking for automatic response",
        f"""
Check out the ticket ticket {ticket_id}. 
If category is Awaiting Details, POST a response requesting more info; otherwise do nothing.
Use JSON: ticket_id={ticket_id}, author="Support Agent", message="<friendly message>"
"""
    )

async def determine_status(ticket_id):
    await run_agent(
        ticket_id,
        "Checking ticket status",
        f"""
Check ticket {ticket_id}. 
If priority is High and the information in the ticket seems urgent, update status to "escalated". Otherwise, do nothing.
"""
    )

async def determine_escalation(ticket_id):
    await run_agent(
        ticket_id,
        "Checking if escalation is required",
        f"""
Check ticket {ticket_id}. 
If status is escalated, POST an automatic escalation response: 
ticket_id={ticket_id}, author="Support Agent", message="<context-aware escalation message>"
"""
    )

async def auto_respond(ticket_id):
    await run_agent(
        ticket_id,
        "Checking if auto-response is needed",
        f"""
Check ticket {ticket_id}'s most recent response. 
If last response is from Support Agent, ignore. Otherwise, POST a placeholder response:
ticket_id={ticket_id}, author="Support Agent", message="<friendly placeholder>"
"""
    )


# --- Async listener loop ---
async for message in listen_for_ticket_updates():
    ticket_id = message.get('ticketId')
    update_type = message.get('updateType')
    print("----------------- MESSAGE -------------")
    print(message)
    print("----------------- Ticket ID -------------")
    print(ticket_id)
    print("----------------- UPDATE TYPE -------------")
    print(update_type)

    if update_type == 'created':
        print("CREATED")
        # Run all ticket handlers asynchronously in sequence
        await determine_category(ticket_id)
        await determine_priority(ticket_id)
        await determine_response(ticket_id)
        await determine_status(ticket_id)
        await determine_escalation(ticket_id)

    elif update_type == 'response':
        await auto_respond(ticket_id)

------------------- Loaded Key -------------------

------------------- Starting Connection and Agent -------------------
WebSocket connection established.
----------------- MESSAGE -------------
{'ticketId': 'd1f411c4-e0bc-4545-88c9-44b3a7031dbe', 'updateType': 'created'}
----------------- Ticket ID -------------
d1f411c4-e0bc-4545-88c9-44b3a7031dbe
----------------- UPDATE TYPE -------------
created
CREATED
Categorizing ticket for ticket: d1f411c4-e0bc-4545-88c9-44b3a7031dbe


> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to get the context of ticket d1f411c4-e0bc-4545-88c9-44b3a7031dbe, determine its category, and then post the assigned category.
Observation: 1. **GET /api/tickets/{ticket_id}**: Retrieve the details of the ticket with ID `d1f411c4-e0bc-4545-88c9-44b3a7031dbe` to get the current context and information about the ticket, including its category.

2. **POST /api/tickets/{ticket_id}/category**: Use the informat

## Basic RAG System
The follow cell sets up a basic Retrieval-Augmented Generation (RAG) retriever for the support information. This allows the agent to access relevant support documents when answering user queries, enhancing its ability to provide accurate and helpful responses. It does this by:

1. Loading the support documents from a specified directory.
2. Creating a vector store to index the documents.
3. Demonstrating how to use the retriever to get relevant information based on a user query.

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

# Initialize the embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Load documents from a directory
loader = DirectoryLoader("./support-info")
docs = loader.load()

# Initialize the vector store and add documents to it
vector_store = InMemoryVectorStore(embeddings)
vector_store.add_documents(docs)

# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Use the retriever to find the most relevant documents for a given query
query = "Machine won't start."
relevant_docs = retriever.get_relevant_documents(query)

# Print the retrieved documents
for doc in relevant_docs:
    print(doc.page_content)